In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [16]:
import torch
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import os
import ast


os.environ['OPENAI_API_KEY']=userdata.get('Open_AI_key')

# Check if cuda is available
use_cuda = torch.cuda.is_available()
# Set proper device based on cuda availability
device = torch.device("cuda" if use_cuda else "cpu")

csv_dir = "/content/drive/MyDrive/class_proj/"

client = OpenAI()
num_prompts = 50
num_loops = 200
prompts = []
for i in range(num_loops):
  try:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant who is an expert at generating interesting prompts \
                                      which are used to generate images using stable diffusion that have only a single object of interest. \
                                      These images are eventually going to be used to generate 3D images, so it is important that only a single object \
                                      exists in each image. Also, the object should be forward facing. Objects can be living things, or inanimate objects. \
                                      Please output the prompts as a nested python list. The format must be [[\"prompt1\"], [\"prompt2\"],..., [\"promptN\"]] \
                                      Do not add anything else to your response so it can be parsed easily for Python code. \
                                      A comma must be after each part of your reply so it can be treated as a list."},
            {
                "role": "user",
                "content": f"Please generate {num_prompts} prompts for use with stable diffusion to provide 2D images as a base to generate 3d images."
            }
        ]
    )

    response = completion.choices[0].message
    response = ast.literal_eval(response.content)
    for prompt in response:
      prompts.append(prompt)
  except:
    print('error')
    continue
prompt_df = pd.DataFrame(prompts, columns=['prompts'])
print(prompt_df.head())
prompt_df.to_csv(csv_dir + 'prompts.csv')

                                             prompts
0              a vibrant red apple on a wooden table
1    a majestic white swan floating on a serene lake
2          a sleek black cat sitting on a windowsill
3  a delicate glass vase filled with colorful flo...
4    a vintage leather suitcase with travel stickers


In [24]:
#Used to concat because prompt lists were short due to errors

df1 = pd.read_csv('/content/drive/MyDrive/class_proj/prompts.csv', index_col=0)
df2 = pd.read_csv('/content/drive/MyDrive/class_proj/prompts2.csv', index_col=0)
df3 = pd.read_csv('/content/drive/MyDrive/class_proj/prompts3.csv', index_col=0)
df2 = df2.rename(columns={'0': 'prompts'})
prompt_final = pd.concat([df1, df2, df3 ], axis=0)
print(prompt_final.head())
prompt_final.reset_index(inplace=True, drop=True)
prompt_final.to_csv(csv_dir + 'prompts_final.csv')

                                     prompts
0     a sleek red sports car, forward facing
1          a vibrant peacock, forward facing
2        a majestic elephant, forward facing
3     a classic wooden chair, forward facing
4  a shiny silver wristwatch, forward facing
